This markdown file is used to preprocess the data starting with the raw data output from the scrapers

In [8]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.engine.base import Engine
import plotly.express as px

# Import the data from the postgres db

In [3]:
def get_db_engine(
    username: str,
    password: str,
    protocol: str = "postgresql",
    server: str = "localhost",
    port: int = 5432,
    dbname: str = "frigg",
) -> Engine:

    engine = create_engine(
        f"{protocol}://" f"{username}:" f"{password}@" f"{server}:" f"{port}/" f"{dbname}",
        isolation_level="AUTOCOMMIT",
    )
    return engine

In [9]:
db_engine = get_db_engine('postgres', 'postgres', dbname='ufc')

In [10]:
with db_engine.connect() as conn:
    bouts = pd.read_sql('SELECT * FROM ufc.bouts', con = conn)
    fighters = pd.read_sql('SELECT * FROM ufc.fighters', con = conn)

## Data pre processing

In [36]:
# First convert bouts date to datetime
bouts['event_date'] = pd.to_datetime(bouts['event_date'], format="%B %d %Y")

In [11]:
fighters.head()


,id,fighter_name,fighter_record,height,weight,reach,stance,date_of_birth,slpm,td_avg,strike_acc,td_acc,sapm,td_def,strike_def,sub_avg
0,1,Hunter Azure,Record: 9-2-0,"5' 8""",145,69,Orthodox,Mar 02 1992,3.92,1.97,53.0,34.0,2.08,67.0,58.0,1.3
1,2,Michael Byrnes,Record: 5-3-0,"5' 11""",155,None,,Jul 17 1990,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0
2,3,Gleidson Cutis,Record: 7-4-0,"5' 9""",155,None,Orthodox,Feb 07 1989,2.99,0.00,52.0,0.0,8.28,0.0,59.0,0.0
3,4,Rolando Dy,Record: 9-7-1 (1 NC),"5' 8""",145,69,Orthodox,Aug 11 1990,3.04,0.30,37.0,20.0,4.47,68.0,52.0,0.0
4,5,Jessica Eye,Record: 15-11-0 (1 NC),"5' 6""",125,66,Orthodox,Jul 27 1986,3.86,0.58,37.0,40.0,4.19,57.0,55.0,0.5


In [25]:
# Filter out the fighters who did not have any fights yet
fighters_that_fought = set(bouts.fighter1).union(set(bouts.fighter2))
fighters = fighters.loc[fighters.fighter_name.isin(fighters_that_fought)]


2297

In [26]:
# Extract total wins and losses
fighters['total_wins'] = fighters.fighter_record.map(lambda x: x.split('Record: ')[1].split('-')[0])
fighters['total_losses'] = fighters.fighter_record.map(lambda x: x.split('Record: ')[1].split('-')[1])
fighters['total_draws'] = fighters.fighter_record.map(lambda x: x.split('Record: ')[1].split('-')[2].split('(')[0])

# Drop fighter_record and index columns
fighters = fighters.drop(['fighter_record'], axis = 1)

AttributeError: 'DataFrame' object has no attribute 'fighter_record'

In [45]:
# Clean date column
fighters.date_of_birth = fighters.date_of_birth.replace('--', None)
fighters['date_of_birth'] = pd.to_datetime(fighters['date_of_birth'], format="%b %d %Y")

/Users/Thijsdebruin/Projects/ufc_fight_predictor/.venv/lib/python3.7/site-packages/pandas/core/missing.py:49: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask = arr == x


,id,fighter_name,height,weight,reach,stance,date_of_birth,slpm,td_avg,strike_acc,td_acc,sapm,td_def,strike_def,sub_avg,total_wins,total_losses,total_draws
0,1,Hunter Azure,"5' 8""",145,69,Orthodox,1992-03-02,3.92,1.97,53.0,34.0,2.08,67.0,58.0,1.3,9,2,0
3,4,Rolando Dy,"5' 8""",145,69,Orthodox,1990-08-11,3.04,0.30,37.0,20.0,4.47,68.0,52.0,0.0,9,7,1
4,5,Jessica Eye,"5' 6""",125,66,Orthodox,1986-07-27,3.86,0.58,37.0,40.0,4.19,57.0,55.0,0.5,15,11,0
6,7,Mike Guymon,"6' 0""",170,74,Orthodox,1974-09-17,2.75,1.87,65.0,37.0,0.79,54.0,66.0,1.2,15,6,1
9,10,Cristiane Justino,"5' 8""",145,68,Orthodox,1985-07-09,7.28,0.66,52.0,55.0,2.25,94.0,64.0,0.4,21,2,0


### Cleaning the height stats
We have some fighters missing height. Our hypothesis is that these missing values only occur for older fighters / fighters early in the ufc.

In [47]:
fighters.loc[fighters.height == '--', :]

,id,fighter_name,height,weight,reach,stance,date_of_birth,slpm,td_avg,strike_acc,td_acc,sapm,td_def,strike_def,sub_avg,total_wins,total_losses,total_draws
206,207,Tulio Palhares,--,180,None,,1990-04-16,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,2,16,0
334,335,Jack Nilson,--,None,None,,1983-12-05,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,2,6,0
523,524,Felix Lee Mitchell,--,None,None,Orthodox,1979-10-24,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,2,7,0
622,623,Jack McGlaughlin,--,None,None,Orthodox,1988-07-14,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,1,4,0
658,659,Francesco Maturi,--,None,None,,1971-01-04,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0,1,0
697,698,Eric Martin,--,None,None,,1989-12-11,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0,1,0
706,707,Cesar Marscucci,--,None,None,,1970-09-15,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,3,2,0
911,912,Jackie Lee,--,None,None,,1982-10-15,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0,1,0
1309,1310,Saeed Hosseini,--,None,None,,1978-08-02,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0,1,0
1387,1388,Noe Hernandez,--,None,None,Orthodox,1986-07-22,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,5,5,0


In [18]:
# Turn height into centimeters, drop all fighters with no listed height
fighters = fighters.loc[fighters.height != '--', :].copy()
fighters['height_feet'] = fighters.height.map(lambda x: int(x.split("' ")[0]))
fighters['height_inch'] = fighters.height.map(lambda x: int(x.split("' ")[1].replace('"', "")))
fighters['height_cm'] = 30.48 * fighters['height_feet'] + 2.54 * fighters['height_inch']
fighters = fighters.drop(['height', 'height_feet', 'height_inch'], axis = 1)

ValueError: invalid literal for int() with base 10: '"5\''

In [16]:
len(fighters)

3780

In [11]:
# check if there are fighters with the same name
fighters[fighters.duplicated(subset="fighter_name", keep=False)]

,id,fighter_name,weight,reach,stance,date_of_birth,slpm,td_avg,strike_acc,td_acc,sapm,td_def,strike_def,sub_avg,total_wins,total_losses,total_draws,height_cm


In [8]:
# note that we have several fighters who have the same names
# Fortunately, they belong to different weight classes
fighters.loc[(fighters.fighter_name=='Michael McDonald') & (fighters.weight==205), "fighter_name"] = 'Michael McDonald 205'
fighters.loc[(fighters.fighter_name=='Tony Johnson') & (fighters.weight==205), "fighter_name"] = 'Tony Johnson 205'
fighters.loc[(fighters.fighter_name=='Joey Gomez') & (fighters.weight==155), "fighter_name"] = 'Joey Gomez 155'
fighters.loc[(fighters.fighter_name=='Mike Davis') & (fighters.weight==145), "fighter_name"] = 'Mike Davis 145'
fighters.loc[(fighters.fighter_name=='Bruno Silva') & (fighters.weight==205), "fighter_name"] = 'Bruno Silva 125'

In [9]:
# Some fighters do not have statistics available, and we will remove those fighters.
fighers = fighters.loc[~((fighters["slpm"] == 0) &
                               (fighters["strike_acc"] == 0) & 
                               (fighters["sapm"] == 0) &
                               (fighters["strike_def"] == 0) &
                               (fighters["td_avg"] == 0) &
                               (fighters["td_acc"] == 0) &
                               (fighters["td_def"] == 0) &
                               (fighters["sub_avg"] == 0))].copy()  
                            

In [10]:
fighters = fighters.loc[fighters['date_of_birth']!='--',:].copy()
fighters.date_of_birth = pd.to_datetime(fighters.date_of_birth)
fighters.head()


,date_of_birth,fighter_name,reach,sapm,slpm,stance,strike_acc,strike_def,sub_avg,td_acc,td_avg,td_def,weight,total_wins,total_losses,total_draws,height_cm
0,1992-03-02,Hunter Azure,69.0,2.08,3.92,Orthodox,53,58.0,1.3,34,1.97,67,145.0,9,2,0,172.72
1,1990-07-17,Michael Byrnes,NaN,0.00,0.00,NaN,0,0.0,0.0,0,0.00,0,155.0,5,3,0,180.34
2,1989-02-07,Gleidson Cutis,NaN,8.28,2.99,Orthodox,52,59.0,0.0,0,0.00,0,155.0,7,4,0,175.26
3,1990-08-11,Rolando Dy,69.0,4.47,3.04,Orthodox,37,52.0,0.0,20,0.30,68,145.0,9,7,1,172.72
4,1986-07-27,Jessica Eye,66.0,4.19,3.96,Orthodox,36,56.0,0.5,44,0.55,57,125.0,15,10,0,167.64


In [31]:
# Get the fighters record in the ufc
def get_ufc_fights(fighter, bouts):
    """Extracts the total number of fights fought in the ufc"""
    bouts_test = bouts.loc[(bouts.fighter1 == fighter) | (bouts.fighter2 == fighter), :].copy()
    wins = len(bouts_test.loc[(bouts_test.winner == fighter) & (bouts_test.win == True), :].copy())
    losses = len(bouts_test.loc[(bouts_test.winner != fighter) & (bouts_test.win == True), :].copy())
    nc = len(bouts_test) - wins - losses
    return wins, losses, nc

fighters[['ufc_wins', 'ufc_losses', 'ufc_nc']] = [get_ufc_fights(fighter, bouts) for fighter in fighters.fighter_name]

In [ ]:
# TODO: Check why we have split decisions in the win = false category
test = bouts.loc[bouts.win == False, :].copy()
test.win_method_type.value_counts()
test.head(100)

In [ ]:
# get columns to merge with the bouts


In [36]:

fighters.head()
bouts_test = bouts.loc[(bouts.fighter1 == 'Hunter Azure') | (bouts.fighter2 == 'Hunter Azure'), :].copy()
bouts_test = bouts_test.sort_values(['event_date'])
bouts_test.head()


,control_time_1,control_time_2,event_date,event_name,fighter1,fighter2,knock_down_1,knock_down_2,performance_bonus,reversals_1,...,takedowns_2,time_minutes,time_seconds,title_fight,total_strike_1,total_strike_2,weight_class,win,win_method_type,winner
32,4:02,5:17,April 10 2021,UFC Fight Night: Vettori vs. Holland,Hunter Azure,Jack Shore,0,0,False,1,...,6,5,0,False,78,57,Bantamweight Bout,True,Decision - Split,Hunter Azure
5852,0:21,0:29,May 13 2020,UFC Fight Night: Smith vs. Teixeira,Hunter Azure,Brian Kelleher,0,1,True,0,...,0,3,40,False,67,32,Featherweight Bout,True,KO/TKO,Hunter Azure
5883,3:13,8:02,September 05 2020,UFC Fight Night: Overeem vs. Sakai,Cole Smith,Hunter Azure,0,1,False,0,...,5,5,0,False,69,75,Bantamweight Bout,True,Decision - Unanimous,Cole Smith
5320,6:22,0:29,September 14 2019,UFC Fight Night: Cowboy vs. Gaethje,Brad Katona,Hunter Azure,0,0,False,0,...,0,5,0,False,40,55,Bantamweight Bout,True,Decision - Unanimous,Brad Katona


## Potential questions for data analysis
Is championship experience a real thing?! win rates when more championship fights
Ring rust
Progression of stopages over time
Duration of bouts over time


